In [1]:
from database import config
import sqlite3 
import pandas as pd
from datetime import datetime

import plotly.graph_objects as go

In [2]:
connection = sqlite3.connect(config.DB_PATH)
df = pd.read_sql_query("SELECT * FROM crypto_OHLCV", connection)

In [3]:
df.set_index('id', inplace=True)

In [4]:
btc = df[df['crypto_id'] == 'BTC']

In [5]:
def convert_unix_to_datetime(date_col):
    'Converts the unix dates into YYYY-MM-DD'
    int_list = list((map(int,date_col)))
    date_list = list(map(datetime.utcfromtimestamp, int_list))
    converted_dates = [date_list[i].strftime('%Y-%m-%d') for i in range(len(date_list))]
    return converted_dates
converted_dates = convert_unix_to_datetime(df['date'])
df['date'] = converted_dates
df = df.replace(r'\n',' ', regex=True)  #removes newline characters from cells

In [6]:
symbol_list = df['crypto_id'].unique()

In [7]:
def plot_crypto(symbol):
    df_symbol = df[df['crypto_id'] == f'{symbol}']
    fig = go.Figure(data=go.Ohlc(x=df_symbol['date'],
                        open=df_symbol['open'],
                        high=df_symbol['high'],
                        low=df_symbol['low'],
                        close=df_symbol['close']))
    fig.update_layout(
    title=f'{symbol} currency')
    return fig.show()

In [16]:
import mplfinance as mpl
import mpld3
from mpld3 import plugins
import matplotlib.pyplot as plt

mpl_df = df.set_index('date')
mpl_df.index = pd.to_datetime(mpl_df.index)

for i in range(1, len(mpl_df.columns)-2):
    mpl_df.iloc[:,i] = list(map(float, mpl_df.iloc[:,i]))


In [31]:
def mpl_crypto(symbol):
    fig = plt.Figure()
    mpl.plot(mpl_df[mpl_df['crypto_id'] == f'{symbol}'])
    plugins.connect(fig)
    return mpld3.show()